<a href="https://colab.research.google.com/github/SchmetterlingIII/D.T./blob/main/Bracing%20Concept/testing/spline_interp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spline Interpretation
The function of this .ipynb file is to:

    a. Interpret the curvature of a 2D spline
    b. Do the same with a 3D spline
    c. Do the same with the sensor data (and from now on, only testing with real data rather than abstracted forms as I am nearing the deadline)

### 2D Spline Interpretation

```
a. Plot these lines & interplolations
b. For each segment, get the second derivative and plot them dynamically
```


In [ ]:
import ipywidgets as ipy
import numpy as np
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt
from IPython.display import display, Math

In [ ]:
def spline_function(x=1):
    fig, ax = plt.subplots(figsize=(5,5))

    x_list = [-10, 0, x]
    eps = 0.01 # small value so that points aren't stacked on top of each other

    # strictly increasing list
    inc_x_list = []
    for i, val in enumerate(x_list):
        if i == 0:
            inc_x_list.append(val)
        else:
            # make sure each value larger than previous
            inc_x_list.append(max(val, inc_x_list[-1] + eps))

    xs = np.array(inc_x_list)
    ys = np.array([0, 50, 100])

    # I cannot interpret the derivatives of clamped cubic splines
    cs = CubicSpline(xs, ys, bc_type='natural')
    coeffs = []
    for i in range(2):
        for j in range(4):
            coeffs.append(cs.c[j,i])
    # coeffs 0 - 3 are for first segment; 4-7 for second
    display(Math(f"f(x) = {coeffs[0]:.5f}(x + {-1 * xs[0]})^3"))
    print(coeffs)
    print(x_list)

    x_plot = np.linspace(xs[0], xs[-1], 100)

    # plots of f(x), f'(x) and f''(x)
    ax.plot(x_plot, cs(x_plot), alpha=0.8, label='f(x)')
    ax.plot(x_plot, cs(x_plot, 1), label = "f'(x)")
    ax.plot(x_plot, cs(x_plot, 2), label = 'f''(x)')
    ax.scatter(xs, ys, color='k')

    """
    if the gradient of the second derivative is greater than three
    i.e. if the third derivative is greater than three
    """
    second_deriv_linear_coeff = coeffs[0] * 6 # gradient of second derivative of first segment
    if second_deriv_linear_coeff > 3:
        ax.clear()
        # plots of f(x), f'(x) and f''(x)
        ax.plot(x_plot, cs(x_plot), color='k', alpha=1, label='f(x)')
        ax.plot(x_plot, cs(x_plot, 1), label = "f'(x)")
        ax.plot(x_plot, cs(x_plot, 2), label = 'f''(x)')
        ax.scatter(xs, ys, color='k')
        print("Threshold has been exceeded.\nPlot not being displayed.")


    # setup
    ax.grid(alpha=0.25)
    ax.set_xlim(-10.5, 10.5)
    ax.set_ylim(-5, 105)

    plt.show()

In [ ]:
ipy.interact(spline_function, x=(0, 10, 0.05))

### Sensor Test
**Deadline: Tomorrow**

Here I will be using the input of the sensor accelerometer data and doing a similar test: first interpolating using a cubic spline and then reading the second derivative in real time.

This can only be done in Thonny so the results of this are to be recorded on my phone.

**Intended Outcome**:
```
1. Live (noisy) sensor data displayed as vectors, using measured positions as inputs (and designing an algorithm to properly handle that)
2. Cubic spline interpolation of this -- still live and handled well
3. Second derivative display of this data, using feedback from how much I have curved the magnetic sheet to define the thresholds for each area.
4. Some output change (flashing lights on the Arduino) when the threshold has been exceeded
```

**Extension:**
```
- Cleaning up noisy sensor data (or displaying this accumulated noise in a graph).
- Seeing how the spine curves naturally (on the sagittal plane) and getting better results through numerical methods (seeing recordings of people moving their spine or something).
    - This can be developed by me using the IMUs as the beginning for good data collection on this. Therefore, if researchers were then to move on the graphene tubes that I am to research, they would have a good baseline to work from.

```

# Challenge Roadmap: Live Spine Model Prototype

### Challenge 1: Live Sensor Data & Kinematic Chaining

**Objective:** To transform raw serial data from multiple IMUs into a live, 3D representation of their positions in space.

-   **Serial to Vector:** Implement a Python script to read and parse the serial stream from the Arduino, converting the raw text into a list of 3D vectors `[(ax1, ay1, az1), (ax2, ay2, az2), ...]`.
-   **Kinematic Chain Algorithm:** Adapt your `forward_kinematics` function to calculate the 3D position of each sensor relative to the one before it, forming a kinematic chain.
-   **Live Visualisation:** Use `matplotlib`'s animation capabilities to create a 3D scatter plot in Thonny that updates in real-time with the calculated sensor positions.



In [ ]:
import serial.tools.list_ports
import string
import serial
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import time

# reading the serial data
BAUDRATE = 115200
try:
    ports = serial.tools.list_ports.comports()
    serialInst = serial.Serial()
    portList = [str(i) for i in ports]
    print(portList)

    com = input("Select COM PORT for Arduino: ")

    for i in range(len(portList)):
        if portList[i].startswith("COM" + str(com)):
            SERIAL_PORT = "COM" + str(com)
            print(SERIAL_PORT)

    serialInst.baudrate = BAUDRATE
    serialInst.port = SERIAL_PORT
    serialInst.open()
    print(f"Connected to {SERIAL_PORT} at {BAUDRATE} baud.")

    ## initial setup: 'begin program'
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line: # if there is data in the readline
            print(f"Arduino: {line}")
        if "Waiting for 'begin program' command" in line:
            break

    print("Sending 'begin' command to Arduino")
    ser.write(b'') # sent in bytes rather than high level strings







except Exception as e:
    print(f"{e}")


### Challenge 2: Real-time Parametric Spline Interpolation

**Objective:** To fit a smooth, 3D curve through the live sensor positions, creating a dynamic model of the spine's shape.

-   **Parametric `CubicSpline`:**
    * **Task:** Use `scipy.interpolate.CubicSpline` instead of `interp1d`. Create three separate spline objects: `cs_x`, `cs_y`, and `cs_z`.
    * **Exploration:** The `t` parameter will be the measured linear distance between your sensors along the spine model (your `linear_distance` list). You will interpolate each axis against this parameter: `cs_x = CubicSpline(t_values, x_positions)`, `cs_y = CubicSpline(t_values, y_positions)`, etc. This is the core of your parametrisation.

-   **Data Handling & Error Management:**
    * **Task:** Your real-time loop (Read -> Calculate -> Interpolate) must be robust. What happens if the Arduino sends a corrupted line of data or a sensor reading spikes unnaturally?
    * **Challenge:** Investigate using `collections.deque` with a fixed length (e.g., `maxlen=5`) to create a moving average for your sensor data. This can smooth out noise and make your spline less erratic. For handling errors (like a missed serial line), implement a `try-except` block to catch parsing errors and decide whether to skip the frame or reuse the last valid data.

-   **Modular Design (Future-Proofing):**
    * **Task:** Structure your code so that the spline type can be easily changed.
    * **Challenge:** Create a main "interpolation" function that takes the spline type as an argument (e.g., `kind='cubic'` or `kind='quadratic'`). This makes it simple to experiment with other spline types later without rewriting your entire script.



### Challenge 3 & 4: Real-time Curvature Analysis & Feedback

**Objective:** To analyze the spline's shape to detect excessive bending and send a feedback signal to the Arduino.

-   **Calculating Curvature from Derivatives:**
    * **Task:** For each spline object, use `cs.derivative(nu=2)` to get the second derivative functions (`x''(t)`, `y''(t)`, `z''(t)`).
    * **Mathematical Challenge:** True curvature ($ \kappa $) is defined by the formula $ \kappa(t)=\frac{\|\mathbf{S}'(t) \times \mathbf{S}''(t)\|}{\|\mathbf{S}'(t)\|^3} $. A powerful and effective proxy for this is the **magnitude of the second derivative vector**:
        $$ \text{Curvature Proxy} = \|\mathbf{S}''(t)\| = \sqrt{(x''(t))^2 + (y''(t))^2 + (z''(t))^2} $$
        Your task is to calculate this value at various points along each segment of your spline in real-time.
    * **Resource:** Your link on deriving curvature (`mathematics.stackexchange.com`) is excellent for understanding the theory. Focus on implementing the proxy formula first, as it directly relates curvature to the second derivatives you can easily compute.

-   **Segment-Specific Thresholding:**
    * **Task:** Create a list or dictionary of threshold values, one for each segment of the spine (e.g., `thresholds = [3.5, 4.0, 4.2, 3.8, 3.5]`).
    * **Challenge:** In your animation loop, check if the calculated curvature proxy for a specific segment exceeds its corresponding threshold.

-   **Arduino Feedback Loop:**
    https://www.youtube.com/watch?v=XXjFtYZEQNw
    * **Task:** When a threshold is breached, use Python's `serial` library to send a specific character (e.g., `'L'`) back to the Arduino.
    * **Challenge:** Modify your Arduino code. It needs to continuously `Serial.read()` to check for incoming data *while* also performing its regular sensor measurements. An `if (Serial.available() > 0)` block will be essential here. When the character `'L'` is received, trigger the onboard LED to flash. This creates the fundamental feedback loop for your haptic motor system later on.